# Базовый TD-WE (Time-Decay Weighted Ensemble)

Ниже приведены:

- **(A)** точная математическая формулировка базовой процедуры,  
- **(B)** теорема о её состоятельности с полным доказательством.

(one–step-ahead горизонт $h=1$; на $h>1$ рассуждения переносятся дословно).

---

## A. Формальное описание алгоритма

### Данные и обозначения

- Имеем вещественный стационарный ряд  
  $$
    (Y_t)_{t\in\mathbb N}.
  $$

- Пусть имеется $M$ базовых моделей  
  $$
    \mathcal M=\{m_1,\dots,m_M\}.
  $$

- Каждая модель порождает точечный прогноз  
  $$
    \hat y_{i,t}=f_i(\mathcal F_t),\qquad  
    \mathcal F_t=\sigma(Y_1,\dots,Y_t).
  $$

- Фиксирована ограниченная функция потерь  
  $$
    \ell:\mathbb R\times\mathbb R\to\mathbb R_+,
    \qquad 0\le\ell\le B<\infty .
  $$

### Роллинговая кросс-валидация

Берём $K$ временных разрезов  

$$
  1<\tau_1<\tau_2<\dots<\tau_K<T,\qquad 
  \tau_k = \tau_0+k\Delta ,
$$

где $\Delta$ — шаг (часто равный сезону).

На $k$-м разрезе:  
- обучаем каждую $m_i$ на $\{1,\dots,\tau_k\}$;  
- строим прогноз $\hat y_{i,\tau_k+1}$;  
- считаем ошибку  
  $$
    \ell_{i,k} =
    \ell\bigl(Y_{\tau_k+1},\,\hat y_{i,\tau_k+1}\bigr).
  $$

### «Свежестные» веса окон  

Параметр затухания $ \lambda\in(0,1] $.

Сырые веса  
$$
  w_k^\star = \lambda^{K-k},
$$

нормировка  
$$
  w_k = \frac{w_k^\star}{\sum_{j=1}^K w_j^\star}.
  \tag{3}
$$

### Агрегированные ошибки моделей  

$$
  E_i = \sum_{k=1}^K w_k\,\ell_{i,k}.
  \tag{4}
$$

### Перевод ошибок в веса моделей  

Берём монотонно убывающую  
$ \varphi:\mathbb R_+\to\mathbb R_+ $:

- обратная пропорция   $ \varphi(x)=1/x$;  
- softmax   $ \varphi(x)=e^{-\beta x},\;\beta>0 $.

Тогда  
$$
  W_i = \frac{\varphi(E_i)}
               {\sum_{j=1}^M \varphi(E_j)},
  \qquad
  W_i\ge 0,\;
  \sum_{i=1}^M W_i=1.
  \tag{5}
$$

### Финальный прогноз  

1. Дообучаем каждую модель на всём отрезке $\{1,\dots,T\}$.
2. Получаем $\hat y_{i,T+1}$.
3. Ансамбль  
   $$
     \hat y^{\text{Ens}}_{T+1} = 
     \sum_{i=1}^M W_i \,\hat y_{i,T+1}.
   \tag{6}
   $$

---

## B. Теорема о состоятельности

### Задача

Для модели $m_i$ обозначим её истинный риск  
$$
  \mu_i = 
  \mathbb E\bigl[\ell(Y_{t+1},f_i(\mathcal F_t))\bigr],
\qquad
  \mu^\star=\min_i \mu_i.
$$

Риск ансамбля:  
$$
  R_{T}^{\text{Ens}} =
  \mathbb E\bigl[\ell(Y_{T+1},\hat y^{\text{Ens}}_{T+1})\bigr].
$$

Показать  
$$
  R_{T}^{\text{Ens}}\;\xrightarrow[T\to\infty]{}\;\mu^\star.
  \tag{7}
$$

### Предпосылки  

- **(H1)** $ (Y_t) $ — α-смешанный стационарный эргодический процесс.  
- **(H2)** Потери ограничены $0\le\ell\le B$.  
- **(H3)** Все прогнозы каузальны (измеримы относительно $\mathcal F_t$).

### Лемма (эрг. закон больших чисел)  

Для любой каузальной функции $g$:  
$$
  \frac1T\sum_{t=1}^T
  \ell\bigl(Y_t,\,g(\mathcal F_{t-1})\bigr)
  \xrightarrow{\text{a.s.}}
  \mathbb E\bigl[\ell(Y_t,g(\mathcal F_{t-1}))\bigr].
$$

### Шаг 1. Сходимость $E_i\to\mu_i$

По лемме и ограниченности потерь:  
$$
  E_i \;\xrightarrow{\text{a.s.}}\;\mu_i.
$$

### Шаг 2. Сходимость весов $W_i$

Пусть  
$$
  \varphi(x)=e^{-\eta x},\quad \eta>0.
$$

Тогда непрерывность $\varphi$ и шаг 1 дают  
$$
  W_i\;\xrightarrow{\text{a.s.}}\;
  W_i^\star=\frac{e^{-\eta\mu_i}}{\sum_j e^{-\eta\mu_j}}.
$$

### Шаг 3. Неасимптотическая граница regret

Классический **Exponentially Weighted Average Forecaster** (EWAF)  
с постоянным $\eta$ удовлетворяет (Cesa-Bianchi & Lugosi, 2006):

$$
  \forall j\;:\quad
  \underbrace{\sum_{t=1}^{T}\!
      \bigl(\ell^{\text{Ens}}_t-\ell_{j,t}\bigr)}_{R_T}
  \;\le\;
  \frac{\ln M}{\eta}
  +\frac{\eta B T}{8}.
  \tag{8}
$$

Оптимальный выбор  
$\displaystyle \eta_T=\sqrt{8\ln M /(B T)}$  
даёт

$$
  \frac{R_T}{T}
  \;\le\;
  \sqrt{\frac{(\ln M)\,B}{2T}}
  \;=\;\mathcal O(T^{-1/2}).
  \tag{9}
$$

Следовательно

$$
  R_{T}^{\text{Ens}}
  \;\le\;
  \mu^\star + \mathcal O(T^{-1/2})
  \;\;\Longrightarrow\;\;
  \lim_{T\to\infty} R_{T}^{\text{Ens}}=\mu^\star .
$$

---

### Теорема (состоятельность)

При условиях **(H1)–(H3)** и выборе  
$\varphi(x)=e^{-\eta x}$
ансамбль (6) удовлетворяет

$$
  \boxed{%
    \displaystyle
    \limsup_{T\to\infty}
    \mathbb E\bigl[\ell(Y_{T+1},\hat y^{\mathrm{Ens}}_{T+1})\bigr]
    \;\le\;\mu^\star
  }
$$

Кроме того, неасимптотическая граница (8) выполняется при любом $T$.

---

### Следствие (концентрация веса на лучшей модели)

Если $\eta_T\to\infty$ достаточно медленно  
(например, $\eta_T=\ln T$),  
то  

$$
  W_i \xrightarrow{\text{a.s.}} \mathbf 1_{\{i=i^\star\}},
$$

то есть веса асимптотически сосредотачиваются  
на действительно оптимальной модели.

---

### Замечания

- Итог остаётся верен для **любой** монотонно-убывающей  
  $\varphi$ с ограниченной липшицевой константой;  
  вместо (8) тогда используется PAC-байесовская оценка.
- Временные веса окон $w_k$ с затуханием $\lambda<1$  
  не влияют на вывод: они образуют вероятностную меру  
  $(\sum w_k=1)$, что достаточно для доказательства EWAF.  
* Для горизонта \(h>1\) можно применять формулу (9)  
  покомпонентно или через суммарную метрику.

